In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("tilsyn.csv", sep=";", dtype={"dato":str})

In [ ]:
df

In [ ]:
df["dato"] = pd.to_datetime(df["dato"], format="%d%m%Y") 

## Oppgaver:
1. Finn gjennomsnittlig antall kontroller pr år
2. Finn spisestedet som har blitt kontrollert flest ganger pr. år - hvilket år var dette i? 
3. Hvilke av de store kjedene (maccern, kurger bing, peppes, pizzabakeren) gjør det best (på tvers av lokasjoner)?
4. Bruk data om kravpunkter til å finne ut hvilke restauranter som har slitt mest med håndvask: https://hotell.difi.no/?dataset=mattilsynet/smilefjes/kravpunkter

#### Oppgave 1

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.year.html
df["year"] = df["dato"].dt.year
df["year"]

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html
# 
# Vi har med "navn"-kolonna her også sånn at vi har noe å telle med .count()
df1gr = df[["year", "navn"]].groupby(by=["year"]).count()
df1gr

In [ ]:
# Kanskje greit å droppe 2015.. ser ut til å ikke ha komplette data
df1gr = df1gr.drop(2015)
df1gr

In [ ]:
df1gr.mean()

#### Oppgave 2

In [ ]:
# Igjen så har vi med tilsynsobjektid sånn at vi har noe å telle (som vi ikke grupperer etter)
dfo2gr = df[["navn", "orgnummer", "year", "tilsynsobjektid"]].groupby(by=["navn", "orgnummer", "year"]).count()
dfo2gr

In [ ]:
dfo2gr = dfo2gr.rename(columns={"tilsynsobjektid":"antall_tilsyn"}).sort_values("antall_tilsyn", ascending=False)
dfo2gr

#### Oppgave 3

In [ ]:
# Litt koderepetisjon her.. men la gå.. 

In [ ]:
# Vi kan se av de unike verdiene at 4 og 5 ikke er i bruk for totalkarakter.
# Disse brukes til å representere at en kontroll ikke er gjennomført 
# 0: smilemunn
# 1: smilemunn, småtterier
# 2: strekmunn
# 3: surmunn
df["total_karakter"].unique()

In [ ]:
maccern = df["navn"].str.lower().str.contains("mcdonald")

In [ ]:
maccern

In [ ]:
maccern.sum()

In [ ]:
burgerking = df["navn"].str.lower().str.contains("burger king")

In [ ]:
burgerking.sum()

In [ ]:
peppes = df["navn"].str.lower().str.contains("peppes pizza")

In [ ]:
peppes.sum()

In [ ]:
pizzabakeren = df["navn"].str.lower().str.contains("pizzabakeren")
pizzabakeren.sum()

In [ ]:
dfo3["kjede"] = None

In [ ]:
dfo3.loc[maccern, "kjede"] = "maccern"
dfo3.loc[burgerking, "kjede"] = "burgerking"
dfo3.loc[peppes, "kjede"] = "peppes"
dfo3.loc[pizzabakeren, "kjede"] = "pizzabakeren"

In [ ]:
dfo3 = dfo3[~dfo3["kjede"].isna()]
dfo3

In [ ]:
# Liten sanity check her
assert dfo3.shape[0] == maccern.sum() + burgerking.sum() + peppes.sum() + pizzabakeren.sum()

In [ ]:
dfo3[["kjede", "total_karakter"]].groupby(by=["kjede"]).mean()

#### Oppgave 4

In [ ]:
df_kp = pd.read_csv("kravpunkter.csv", sep=";", dtype={"dato":str})

In [ ]:
df_kp

In [ ]:
df_kp["kravpunktnavn_no"].unique()

In [ ]:
df_handvask = df_kp[(df_kp["kravpunktnavn_no"] == "Håndvask") & (df_kp["karakter"] == 3)]
df_handvask

In [ ]:
handvask_3 = df[["orgnummer", "navn", "dato", "poststed", "tilsynid"]].set_index("tilsynid").join(
    df_handvask[["tilsynid", "karakter", "tekst_no"]].set_index("tilsynid"), how="inner")
handvask_3

In [ ]:
handvask_3.reset_index(drop=True).sort_values("dato", ascending=False)